# Crypto Portfolio Analysis

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from binance import Client

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

# Import panel.interact
from panel.interact import interact

# for merging the dataframes
from functools import reduce

## Instantiate the Binance API Client

In [ ]:
# Pulling price data doesn't require an api key as it is publicly available
# https://python-binance.readthedocs.io/en/latest/
# https://binance-docs.github.io/apidocs/spot/en/#compressed-aggregate-trades-list
client = Client()

## Get list of all cryptocurrencies traded on the Binance exchange


In [ ]:
exchange_info = client.get_exchange_info()
symbols = exchange_info['symbols']
# Ensure no duplicates by using sets
coin_set = {s['baseAsset'] for s in symbols} | {s['quoteAsset'] for s in symbols}
coin_list = sorted(coin_set)

## Create a DataFrame of the closing prices and dates for each chosen cryptocurrency in the portfolio

In [ ]:
def get_historical_data(currency):
    klines = client.get_historical_klines(currency + 'USDT', Client.KLINE_INTERVAL_1DAY, "1 year ago UTC")
    # klines columns=['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df = pd.DataFrame(((x[0], x[4]) for x in klines), columns=['timestamp', currency])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

# Chosen cryptocurrenies
crypto_portfolio = ('BTC', 'ETH', 'SOL')
# Global dataframe
df = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), map(get_historical_data, crypto_portfolio))
df.head()

- - -

## Panel Visualisation

In [ ]:
# Make your plot here
